## Using ML anonymization to defend against attribute inference attacks

### Load data

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

import warnings
# Filter out all warnings
warnings.filterwarnings("ignore")


#### First of all, we need to import the required packages to perform our privacy analysis and mitigation. You will need to have the `holisticai` package installed on your system, remember that you can install it by running: 
```bash
!pip install holisticai[all]
```

In [2]:
from holisticai.datasets import load_dataset
loaded = load_dataset(dataset='adult', preprocessed=False, as_array=False)
df = pd.DataFrame(data=loaded.data, columns=loaded.feature_names)
df['class'] = loaded.target.apply(lambda x: 1 if x == '>50K' else 0)

### Computing k-Anonymity metric 

In [3]:
from holisticai.privacy.metrics import k_anonymity

QI = ['education', 'marital-status', 'age']

k_anon = k_anonymity(df, qi=QI)

k_anon.head(15)


education     marital-status      age 
Some-college  Never-married       20.0    600
                                  21.0    545
                                  19.0    484
                                  22.0    423
HS-grad       Never-married       19.0    368
Some-college  Never-married       23.0    351
HS-grad       Never-married       20.0    311
                                  21.0    305
                                  22.0    285
                                  23.0    272
11th          Never-married       17.0    266
Bachelors     Never-married       23.0    263
11th          Never-married       18.0    254
Bachelors     Never-married       24.0    252
HS-grad       Married-civ-spouse  35.0    246
Name: count, dtype: int64

### Computing l-Diversity metric

In [8]:
from holisticai.privacy.metrics import l_diversity

QI = ['education', 'marital-status']
sensitive_attribute = ['race']

l_div = l_diversity(df, qi=QI, sa=sensitive_attribute)

l_div

{'race': [1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5]}

### Preparing dataset 

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical features
categorical_features = X.select_dtypes(include=['category']).columns

# Create transformers for numerical and categorical features
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine transformers into a preprocessor using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, X.select_dtypes(exclude=['category']).columns),
        ('cat', categorical_transformer, categorical_features)
    ])

# Fit and transform your data using the ColumnTransformer
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

### Train decision tree model

In [12]:
from sklearn.tree import DecisionTreeClassifier

DTC = DecisionTreeClassifier()
DTC.fit(X_train_transformed, y_train)
# Predict values
y_pred = DTC.predict(X_test_transformed)
y_proba = DTC.predict_proba(X_test_transformed)
print('Base model accuracy: ', DTC.score(X_test_transformed, y_test))

Base model accuracy:  0.8212713686150066


### BlackBox Attack

In [13]:
from holisticai.privacy.metrics import BlackBoxAttack

attack_feature = 'education'
predictions_of_attack_feature_1 = BlackBoxAttack(attack_feature, X_train, y_train, X_test, y_test)
print(accuracy_score(X_test['education'], predictions_of_attack_feature_1))


0.8921076875831713


#### This means that for 89% of the training set, the attacked feature is inferred correctly using this attack.

